In [25]:
import pandas as pd
import numpy as np
import pickle

csv_file = "motions/unitree_rl_lab/G1_Take_102.bvh_60hz.csv"
ouotput_file = "motions/unitree_rl_lab/G1_Take_102.bvh_60hz.pkl"

motion_datas = pd.read_csv(csv_file, header=None).to_numpy()
# motion_datas = pd.read_csv(csv_file, header=None)


link_body_list = ['pelvis', 'left_hip_pitch_link', 'left_hip_roll_link', 'left_hip_yaw_link', 'left_knee_link', 'left_ankle_pitch_link', 'left_ankle_roll_link', 
                  'left_toe_link', 'pelvis_contour_link', 'right_hip_pitch_link', 'right_hip_roll_link', 'right_hip_yaw_link', 'right_knee_link', 
                  'right_ankle_pitch_link', 'right_ankle_roll_link', 'right_toe_link', 'waist_yaw_link', 'waist_roll_link', 'torso_link', 'head_link', 
                  'head_mocap', 'imu_in_torso', 'left_shoulder_pitch_link', 'left_shoulder_roll_link', 'left_shoulder_yaw_link', 'left_elbow_link', 
                  'left_wrist_roll_link', 'left_wrist_pitch_link', 'left_wrist_yaw_link', 'left_rubber_hand', 'right_shoulder_pitch_link', 'right_shoulder_roll_link', 
                  'right_shoulder_yaw_link', 'right_elbow_link', 'right_wrist_roll_link', 'right_wrist_pitch_link', 'right_wrist_yaw_link', 'right_rubber_hand']

print(motion_datas.shape)
# print(motion_datas)

motion_dicts = {
    'fps': np.float64(60),
    'root_pos' : motion_datas[:, 0:3],
    'root_rot' : motion_datas[:, 3:7],
    'dof_pos' : motion_datas[:, 7:],
    'local_body_pos': np.random.randn(len(motion_datas), 38, 3).astype(np.float32),
    'link_body_list': link_body_list

}

with open(ouotput_file, 'wb') as f:
    pickle.dump(motion_dicts, f)


(1749, 36)


In [ ]:
import pickle

def load_pkl_file(file_path):
    """
    读取pkl文件并返回其中的数据
    
    Args:
        file_path (str): pkl文件的路径
    
    Returns:
        any: pkl文件中保存的Python对象（可能是列表、字典、类实例等）
    
    Raises:
        FileNotFoundError: 文件路径不存在时抛出
        pickle.UnpicklingError: 文件损坏或不是合法的pkl文件时抛出
    """
    try:
        # 以二进制读取模式打开文件
        with open(file_path, 'rb') as f:
            # 加载pkl文件中的数据
            data = pickle.load(f)
        return data
    except FileNotFoundError:
        print(f"错误：找不到文件 {file_path}")
        raise
    except pickle.UnpicklingError:
        print(f"错误：{file_path} 不是有效的pkl文件或文件已损坏")
        raise
    except Exception as e:
        print(f"读取文件时发生未知错误：{e}")
        raise

# 示例用法
if __name__ == "__main__":
    # 替换为你的pkl文件路径
    pkl_file_path = "/home/hdh/work/project_rl_Lab/motions/example_motions/0807_yanjie_walk_003.pkl"
    try:
        # 读取pkl文件
        pkl_data = load_pkl_file(pkl_file_path)
        
        # 打印数据类型和内容（方便查看）
        print(f"pkl文件中的数据类型：{type(pkl_data)}")
        print("pkl文件中的数据内容：")
        print(pkl_data)
        
        # 如果是字典/列表等可迭代对象，可以进一步解析
        if isinstance(pkl_data, dict):
            print("\n解析字典的键：", list(pkl_data.keys()))
        elif isinstance(pkl_data, list):
            print(f"\n列表长度：{len(pkl_data)}，前5个元素：{pkl_data[:5]}")
        print(pkl_data['link_body_list'])
    except Exception as e:
        print(f"执行失败：{e}")


In [27]:
import pickle
import sys
import numpy as np  # 新增：导入numpy库
from typing import Any, Dict, List, Tuple

def _get_type_name(obj: Any) -> str:
    """获取对象的类型名称（简化显示）"""
    # 优先判断numpy数组类型
    if isinstance(obj, np.ndarray):
        return "numpy.ndarray"
    
    type_name = str(type(obj)).split("'")[1]
    # 简化内置类型名称
    builtin_types = {
        'dict': 'dict',
        'list': 'list',
        'tuple': 'tuple',
        'set': 'set',
        'int': 'int',
        'float': 'float',
        'str': 'str',
        'bool': 'bool',
        'NoneType': 'None'
    }
    return builtin_types.get(type_name, type_name)

def _format_value(obj: Any) -> str:
    """格式化值的显示（避免超长内容）"""
    # 新增：处理numpy数组，返回维度信息
    if isinstance(obj, np.ndarray):
        return f"shape={obj.shape}, dtype={obj.dtype}"
    
    if isinstance(obj, str):
        # 字符串最多显示前20个字符
        return f"'{obj[:20]}...'" if len(obj) > 20 else f"'{obj}'"
    elif isinstance(obj, (int, float, bool)) or obj is None:
        return str(obj)
    else:
        return ''

def print_pkl_tree(
    obj: Any,
    prefix: str = '',
    is_last: bool = True,
    visited: set = None,
    max_depth: int = 10,
    current_depth: int = 0
) -> None:
    """
    递归打印对象的树状结构
    
    Args:
        obj: 要打印的对象
        prefix: 层级前缀符号
        is_last: 是否是当前层级最后一个元素
        visited: 记录已访问的对象（防止循环引用）
        max_depth: 最大递归深度（避免无限递归）
        current_depth: 当前递归深度
    """
    # 初始化已访问集合（防止循环引用）
    if visited is None:
        visited = set()
    
    # 防止循环引用和过深递归
    obj_id = id(obj)
    if obj_id in visited or current_depth >= max_depth:
        if obj_id in visited:
            print(f"{prefix}{'└──' if is_last else '├──'} [循环引用] {_get_type_name(obj)}")
        else:
            print(f"{prefix}{'└──' if is_last else '├──'} [超过最大深度] {_get_type_name(obj)}")
        return
    visited.add(obj_id)
    
    # 新增：专门处理numpy数组（基础类型判断前优先处理）
    if isinstance(obj, np.ndarray):
        type_name = _get_type_name(obj)
        value = _format_value(obj)
        print(f"{prefix}{'└──' if is_last else '├──'} {type_name}: {value}")
        return
    
    # 基础类型直接打印
    basic_types = (int, float, str, bool, type(None))
    if isinstance(obj, basic_types):
        type_name = _get_type_name(obj)
        value = _format_value(obj)
        print(f"{prefix}{'└──' if is_last else '├──'} {type_name}: {value}")
        return
    
    # 容器类型递归处理
    type_name = _get_type_name(obj)
    # 打印当前节点
    print(f"{prefix}{'└──' if is_last else '├──'} {type_name}")
    
    # 更新前缀（为子节点准备）
    child_prefix = prefix + ('    ' if is_last else '│   ')
    
    # 处理字典
    if isinstance(obj, dict):
        items = list(obj.items())
        for i, (key, value) in enumerate(items):
            is_child_last = (i == len(items) - 1)
            # 打印键
            print(f"{child_prefix}{'└──' if is_child_last else '├──'} '{key}':")
            # 递归打印值
            print_pkl_tree(
                value,
                prefix=child_prefix + ('    ' if is_child_last else '│   '),
                is_last=is_child_last,
                visited=visited,
                max_depth=max_depth,
                current_depth=current_depth + 1
            )
    
    # 处理列表/元组/集合
    elif isinstance(obj, (list, tuple, set)):
        items = list(obj)
        # 限制显示的元素数量（避免超大列表刷屏）
        max_items = 20
        if len(items) > max_items:
            items = items[:max_items] + [f"[剩余 {len(items)-max_items} 个元素]"]
        
        for i, item in enumerate(items):
            is_child_last = (i == len(items) - 1)
            print_pkl_tree(
                item,
                prefix=child_prefix,
                is_last=is_child_last,
                visited=visited,
                max_depth=max_depth,
                current_depth=current_depth + 1
            )
    
    # 处理自定义类实例
    else:
        try:
            # 获取实例的属性（过滤内置属性）
            attrs = {k: v for k, v in obj.__dict__.items() if not k.startswith('__')}
            if attrs:
                items = list(attrs.items())
                for i, (key, value) in enumerate(items):
                    is_child_last = (i == len(items) - 1)
                    print(f"{child_prefix}{'└──' if is_child_last else '├──'} .{key}:")
                    print_pkl_tree(
                        value,
                        prefix=child_prefix + ('    ' if is_child_last else '│   '),
                        is_last=is_child_last,
                        visited=visited,
                        max_depth=max_depth,
                        current_depth=current_depth + 1
                    )
            else:
                print(f"{child_prefix}└── [无自定义属性]")
        except:
            print(f"{child_prefix}└── [无法解析实例属性]")

def main(pkl_file_path: str, max_depth: int = 10):
    """
    主函数：读取pkl文件并打印树状结构
    
    Args:
        pkl_file_path: pickle文件路径
        max_depth: 最大递归深度（默认10）
    """
    try:
        # 读取pickle文件
        with open(pkl_file_path, 'rb') as f:
            data = pickle.load(f)
        
        print(f"\n=== Pickle文件 {pkl_file_path} 的树状结构 ===")
        print_pkl_tree(data, max_depth=max_depth)
        print("\n=== 打印完成 ===")
    
    except FileNotFoundError:
        print(f"错误：文件 {pkl_file_path} 不存在！")
        sys.exit(1)
    except pickle.UnpicklingError:
        print(f"错误：文件 {pkl_file_path} 不是有效的pickle文件！")
        sys.exit(1)
    except Exception as e:
        print(f"错误：读取/解析文件时发生异常 - {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    # 命令行参数处理
    
    # pkl_path = "/home/hdh/work/project_rl_Lab/motions/example_motions/0807_yanjie_walk_003.pkl"
    # pkl_path = "/home/hdh/work/project_rl_Lab/motions/unitree_rl_lab/G1_Take_102.bvh_60hz.pkl"
    pkl_path = "/home/hdh/work/project_rl_Lab/motions/unitree_rl_lab/G1_gangnam_style_V01.bvh_60hz.pkl"
    max_depth = int(sys.argv[2]) if len(sys.argv) >= 3 else 10
    main(pkl_path, max_depth)



=== Pickle文件 /home/hdh/work/project_rl_Lab/motions/unitree_rl_lab/G1_gangnam_style_V01.bvh_60hz.pkl 的树状结构 ===
└── dict
    ├── 'fps':
    │   ├── numpy.float64: 60.0
    ├── 'root_pos':
    │   ├── numpy.ndarray: shape=(1942, 3), dtype=float64
    ├── 'root_rot':
    │   ├── numpy.ndarray: shape=(1942, 4), dtype=float64
    ├── 'dof_pos':
    │   ├── numpy.ndarray: shape=(1942, 29), dtype=float64
    ├── 'local_body_pos':
    │   ├── numpy.ndarray: shape=(1942, 38, 3), dtype=float32
    └── 'link_body_list':
        └── list
            ├── str: 'pelvis'
            ├── str: 'left_hip_pitch_link'
            ├── str: 'left_hip_roll_link'
            ├── str: 'left_hip_yaw_link'
            ├── str: 'left_knee_link'
            ├── str: 'left_ankle_pitch_lin...'
            ├── str: 'left_ankle_roll_link'
            ├── str: 'left_toe_link'
            ├── str: 'pelvis_contour_link'
            ├── str: 'right_hip_pitch_link'
            ├── str: 'right_hip_roll_link'
            ├── 